In [1]:
import pandas as pd
import numpy as np

In [2]:
# Replace with the path to your Excel file
file_path = '/home/ayarkhan01/Desktop/Projects/Financial Model/data.xlsx'

# Load the data
df = pd.read_excel(file_path)

In [3]:
# Convert all columns except the first one to datetime
df.columns = ['Customer'] + [pd.to_datetime(col) for col in df.columns[1:]]

# Apply strftime to format the datetime columns
df.columns = ['Customer'] + [col.strftime('%b %Y') for col in df.columns[1:]]


In [4]:
# Drop 'Customer' columns from the original DataFrame
df = df.drop(columns=['Customer'])

In [5]:
# Shift each row to the right to help with calculations
shifted_df = df.shift(1, axis=1, fill_value=0)

In [6]:
# Calculate new customers summary
new_customers = (shifted_df == 0) & (df != 0)
new_customers_count = new_customers.sum()
new_customer_sums = df[new_customers].sum() * 12
new_summary_df = pd.DataFrame([new_customers_count, new_customer_sums], index=['new_customers', 'new_revenue'])

In [7]:
# Calculate churned customers summary
churn_customers = (shifted_df != 0) & (df == 0)
churn_customer_counts = churn_customers.sum()
churn_customer_sums = shifted_df[churn_customers].sum() * 12
churn_summary_df = pd.DataFrame([churn_customer_counts, churn_customer_sums], index=['churned_customers', 'churned_revenue'])

In [8]:
# Combine the summaries
combined_summary_df = pd.concat([new_summary_df, churn_summary_df])

In [9]:
combined_summary_df

,Jan 2020,Feb 2020,Mar 2020,Apr 2020,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,...,May 2022,Jun 2022,Jul 2022,Aug 2022,Sep 2022,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Jan 1970
new_customers,13.0,0.0,4.0,1.0,0.0,3.0,2.0,2.0,2.0,2.0,...,6.0,13.0,4.0,7.0,10.0,8.0,8.0,15.0,14.0,8.0
new_revenue,353445.0,0.0,170000.0,50000.0,0.0,124000.0,40000.0,55000.0,65000.0,40000.0,...,186500.0,515245.0,140000.0,252500.0,376000.0,276000.0,277000.0,533080.0,485335.0,280850.0
churned_customers,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0
churned_revenue,0.0,0.0,0.0,0.0,0.0,50000.0,0.0,0.0,0.0,0.0,...,0.0,49000.0,50000.0,25000.0,25000.0,30000.0,0.0,47500.0,90000.0,0.0


In [10]:
# Calculate upsell summary
upsell_customers = (shifted_df != 0) & (shifted_df < df)
upsell_values = (df - shifted_df).where(upsell_customers, 0)
upsell_sums_annualized = upsell_values.sum() * 12
upsell_summary_df = pd.DataFrame([upsell_sums_annualized], index=['upsell'])
combined_summary_df = pd.concat([combined_summary_df, upsell_summary_df])

In [11]:
# Calculate downgrade summary
downgrade_customers = (df != 0) & (shifted_df > df)
downgrade_values = (df - shifted_df).where(downgrade_customers, 0)
downgrade_sums_annualized = downgrade_values.sum() * 12
downgrade_summary_df = pd.DataFrame([downgrade_sums_annualized], index=['downgrade'])
combined_summary_df = pd.concat([combined_summary_df, downgrade_summary_df])

In [12]:
# Calculate non-zero counts for original DataFrame
non_zero_counts = df[df != 0].count()
customers = pd.DataFrame(non_zero_counts).transpose()
customers.index = ['Customers']

In [13]:
# Calculate YoY growth
original_columns = customers.columns
df_t = customers.transpose()
df_t.index = pd.to_datetime(df_t.index, format='%b %Y')
df_t['YoY Growth'] = df_t['Customers'].pct_change(periods=12) * 100
df_t.index = original_columns
df_final = df_t.transpose()

In [14]:
# Calculate ARPA (Average Revenue Per Account)
sum_of_months = df.sum()
total_mrr = pd.DataFrame(sum_of_months).transpose()
total_mrr.index = ['MRR']
aligned_mrr = total_mrr.loc['MRR']
aligned_customers = customers.loc['Customers']
result = aligned_mrr / aligned_customers
ARPA_Monthly = pd.DataFrame(result).transpose()
ARPA_Monthly.index = ['ARPA (monthly)']

In [15]:
# Calculate YoY growth for ARPA
original_columns = ARPA_Monthly.columns
df_t = ARPA_Monthly.transpose()
df_t.index = pd.to_datetime(df_t.index, format='%b %Y')
df_t['YoY Growth'] = df_t['ARPA (monthly)'].pct_change(periods=12) * 100
df_t.index = original_columns
df_final_ARPA_Monthly = df_t.transpose()

In [16]:
# Combine customer analysis data
Customer_analysis_1 = pd.concat([df_final, df_final_ARPA_Monthly])

In [17]:
# Create new DataFrame for customer analysis
index_names = ['BoP', 'New Customers', 'Churned Customers', 'EoP', 'Monthly Logo Churn', 'Implied Customer Lifetime', 'Annualized Logo Churn', 'LTM Logo Churn', 'L6M New Customers', 'Average L6M New Customers']
new_df = pd.DataFrame(index=index_names, columns=df.columns)
new_df.loc['New Customers', :] = combined_summary_df.loc['new_customers', :]
new_df.loc['Churned Customers', :] = combined_summary_df.loc['churned_customers', :]
new_df.iloc[3, 0] = df_final.iloc[0, 0]

In [18]:
# Calculate various metrics
first_column = new_df.columns[0]
for idx, column in enumerate(new_df.columns):
    if idx == 0:
        continue
    prev_column = new_df.columns[idx - 1]
    new_df.loc["BoP", column] = new_df.loc["EoP", prev_column]
    new_df.loc["EoP", column] = new_df.loc["BoP", column] + new_df.loc["New Customers", column] - new_df.loc["Churned Customers", column]
    new_df.loc["Monthly Logo Churn", column] = new_df.loc["Churned Customers", column] / new_df.loc["BoP", column] * 100
    new_df.loc["Annualized Logo Churn", column] = new_df.loc["Monthly Logo Churn", column] * 12
    new_df.loc['Implied Customer Lifetime', column] = (1 / new_df.loc['Monthly Logo Churn', first_column:column].mean()) / 12 * 100

    if idx < 6:
        continue

    l6m_columns = new_df.columns[idx - 5:idx + 1]
    new_df.loc["L6M New Customers", column] = new_df.loc['New Customers', l6m_columns].sum()
    new_df.loc["Average L6M New Customers", column] = new_df.loc["L6M New Customers", l6m_columns].mean()

    if idx < 12:
        continue

    l12m_columns = new_df.columns[idx - 11:idx + 1]
    new_df.loc["LTM Logo Churn", column] = (new_df.loc['Churned Customers', l12m_columns].sum()) / new_df.loc['BoP', new_df.columns[idx - 11]] * 100

Customer_analysis_2 = new_df

/tmp/ipykernel_14013/3861825215.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  new_df.loc['Implied Customer Lifetime', column] = (1 / new_df.loc['Monthly Logo Churn', first_column:column].mean()) / 12 * 100


In [19]:
# Combine all customer analysis data
Customer_analysis = pd.concat([Customer_analysis_1, Customer_analysis_2])

In [27]:
Customer_analysis

,Jan 2020,Feb 2020,Mar 2020,Apr 2020,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,...,May 2022,Jun 2022,Jul 2022,Aug 2022,Sep 2022,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Jan 1970
Customers,13.0,13.0,17.0,18.0,18.0,20.0,22.0,24.0,26.0,28.0,...,101.0,113.0,116.0,122.0,131.0,138.0,146.0,159.0,171.0,179.0
YoY Growth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,152.5,135.416667,118.867925,106.779661,98.484848,100.0,100.0,101.265823,113.75,110.588235
ARPA (monthly),2265.673077,2746.442308,3178.651961,3323.819444,3509.00463,3466.4375,3302.82197,3270.642361,3259.439103,3145.669643,...,3387.929043,3449.421091,3516.282328,3575.317623,3587.954835,3579.870169,3591.589612,3619.165094,3576.316764,3557.705773
YoY Growth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.901428,9.911603,8.96056,8.993964,12.121598,11.33122,12.508292,9.011516,7.34502,6.497193
BoP,NaN,13.0,13.0,17.0,18.0,18.0,20.0,22.0,24.0,26.0,...,95.0,101.0,113.0,116.0,122.0,131.0,138.0,146.0,159.0,171.0
New Customers,13.0,0.0,4.0,1.0,0.0,3.0,2.0,2.0,2.0,2.0,...,6.0,13.0,4.0,7.0,10.0,8.0,8.0,15.0,14.0,8.0
Churned Customers,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0
EoP,13.0,13.0,17.0,18.0,18.0,20.0,22.0,24.0,26.0,28.0,...,101.0,113.0,116.0,122.0,131.0,138.0,146.0,159.0,171.0,179.0
Monthly Logo Churn,NaN,0.0,0.0,0.0,0.0,5.555556,0.0,0.0,0.0,0.0,...,0.0,0.990099,0.884956,0.862069,0.819672,0.763359,0.0,1.369863,1.257862,0.0
Implied Customer Lifetime,NaN,inf,inf,inf,inf,7.5,9.0,10.5,12.0,13.5,...,8.776973,8.764036,8.78433,8.810271,8.847157,8.898274,9.167919,9.036997,8.946519,9.195034


In [20]:
combined_summary_df

,Jan 2020,Feb 2020,Mar 2020,Apr 2020,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,...,May 2022,Jun 2022,Jul 2022,Aug 2022,Sep 2022,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Jan 1970
new_customers,13.0,0.0,4.0,1.0,0.0,3.0,2.0,2.0,2.0,2.0,...,6.0,13.0,4.0,7.0,10.0,8.0,8.0,15.0,14.0,8.0
new_revenue,353445.0,0.0,170000.0,50000.0,0.0,124000.0,40000.0,55000.0,65000.0,40000.0,...,186500.0,515245.0,140000.0,252500.0,376000.0,276000.0,277000.0,533080.0,485335.0,280850.0
churned_customers,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0
churned_revenue,0.0,0.0,0.0,0.0,0.0,50000.0,0.0,0.0,0.0,0.0,...,0.0,49000.0,50000.0,25000.0,25000.0,30000.0,0.0,47500.0,90000.0,0.0
upsell,0.0,75000.0,50000.0,19500.0,40000.0,0.0,0.0,15000.0,10000.0,0.0,...,74450.0,105000.0,127250.0,112100.0,55000.0,42000.0,87200.0,127322.0,62900.0,32500.0
downgrade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25000.0,-10000.0


In [21]:
arr_index_names = [
    'BoP', 'New', 'Upsell', 'Downgrades', 'Churn', 'EoP',
    'Monthly Gross Churn (w/ Downsell)', 'Annualized Gross Churn (w/ Downsell)', 'LTM Gross Churn (w/ Downsell)',
    'Monthly Gross Churn', 'Annualized Gross Churn', 'LTM Gross Churn',
    'Monthly Net Retention', 'LTM Net Retention'
]

# Initialize arr_analysis DataFrame with the specified indexes and columns from the original DataFrame
arr_analysis = pd.DataFrame(index=arr_index_names, columns=df.columns)

In [22]:
arr_analysis.iloc[5, 0] = combined_summary_df.iloc[1, 0]

In [23]:
df_transposed = combined_summary_df

In [25]:
for idx, column in enumerate(arr_analysis.columns):
    if idx == 0:
        continue
    prev_column = arr_analysis.columns[idx - 1]
    arr_analysis.loc["BoP", column] = arr_analysis.loc['EoP', prev_column]
    arr_analysis.loc["New", column] = df_transposed.loc["new_revenue", column]
    arr_analysis.loc["Upsell", column] = df_transposed.loc["upsell", column]
    arr_analysis.loc['Downgrades', column] = df_transposed.loc["downgrade", column]
    arr_analysis.loc['Churn', column] = -df_transposed.loc["churned_revenue", column]
    arr_analysis.loc["EoP", column] = arr_analysis.loc["BoP", column] + arr_analysis.loc["New", column]+ arr_analysis.loc["Upsell", column] + arr_analysis.loc["Churn", column] + arr_analysis.loc["Downgrades", column]
    arr_analysis.loc["Monthly Gross Churn (w/ Downsell)", column] = (arr_analysis.loc['Downgrades', column] + arr_analysis.loc['Churn', column]) / arr_analysis.loc["BoP", column]
    arr_analysis.loc["Annualized Gross Churn (w/ Downsell)", column] = arr_analysis.loc["Monthly Gross Churn (w/ Downsell)", column] * 12
    arr_analysis.loc["Monthly Gross Churn", column] = arr_analysis.loc['Churn', column] / arr_analysis.loc["BoP", column]
    arr_analysis.loc["Annualized Gross Churn", column] = arr_analysis.loc["Monthly Gross Churn", column] * 12
    arr_analysis.loc["Monthly Net Retention", column] = (arr_analysis.loc["BoP", column] + arr_analysis.loc["Upsell", column] + arr_analysis.loc['Downgrades', column] + arr_analysis.loc['Churn', column])/arr_analysis.loc["BoP", column]
    
    if idx < 12:
        continue
    l12m_columns = arr_analysis.columns[idx - 11:idx + 1]
    arr_analysis.loc["LTM Gross Churn (w/ Downsell)", column] = (arr_analysis.loc['Churn', l12m_columns].sum() + arr_analysis.loc['Downgrades', l12m_columns].sum()) / arr_analysis.loc['BoP', arr_analysis.columns[idx - 11]]
    arr_analysis.loc["LTM Gross Churn", column] = (arr_analysis.loc['Churn', l12m_columns].sum()) / arr_analysis.loc['BoP', arr_analysis.columns[idx - 11]]
    arr_analysis.loc["LTM Net Retention", column] = (arr_analysis.loc['Churn', l12m_columns].sum() + arr_analysis.loc['Downgrades', l12m_columns].sum() + arr_analysis.loc['Upsell', l12m_columns].sum() + arr_analysis.loc['BoP', arr_analysis.columns[idx - 11]] )/ arr_analysis.loc['BoP', arr_analysis.columns[idx - 11]]


In [26]:
arr_analysis

,Jan 2020,Feb 2020,Mar 2020,Apr 2020,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,...,May 2022,Jun 2022,Jul 2022,Aug 2022,Sep 2022,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Jan 1970
BoP,NaN,353445.0,428445.0,648445.0,717945.0,757945.0,831945.0,871945.0,941945.0,1016945.0,...,3845220.0,4106170.0,4677415.0,4894665.0,5234265.0,5640265.0,5928265.0,6292465.0,6905367.0,7338602.0
New,NaN,0.0,170000.0,50000.0,0.0,124000.0,40000.0,55000.0,65000.0,40000.0,...,186500.0,515245.0,140000.0,252500.0,376000.0,276000.0,277000.0,533080.0,485335.0,280850.0
Upsell,NaN,75000.0,50000.0,19500.0,40000.0,0.0,0.0,15000.0,10000.0,0.0,...,74450.0,105000.0,127250.0,112100.0,55000.0,42000.0,87200.0,127322.0,62900.0,32500.0
Downgrades,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25000.0,-10000.0
Churn,NaN,-0.0,-0.0,-0.0,-0.0,-50000.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-49000.0,-50000.0,-25000.0,-25000.0,-30000.0,-0.0,-47500.0,-90000.0,-0.0
EoP,353445.0,428445.0,648445.0,717945.0,757945.0,831945.0,871945.0,941945.0,1016945.0,1056945.0,...,4106170.0,4677415.0,4894665.0,5234265.0,5640265.0,5928265.0,6292465.0,6905367.0,7338602.0,7641952.0
Monthly Gross Churn (w/ Downsell),NaN,0.0,0.0,0.0,0.0,-0.065968,0.0,0.0,0.0,0.0,...,0.0,-0.011933,-0.01069,-0.005108,-0.004776,-0.005319,0.0,-0.007549,-0.016654,-0.001363
Annualized Gross Churn (w/ Downsell),NaN,0.0,0.0,0.0,0.0,-0.791614,0.0,0.0,0.0,0.0,...,0.0,-0.143199,-0.128276,-0.061291,-0.057315,-0.063827,0.0,-0.090585,-0.199845,-0.016352
LTM Gross Churn (w/ Downsell),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.244476,-0.227223,-0.224488,-0.194084,-0.173904,-0.176811,-0.148313,-0.139721,-0.173449,-0.165739
Monthly Gross Churn,NaN,-0.0,-0.0,-0.0,-0.0,-0.065968,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.011933,-0.01069,-0.005108,-0.004776,-0.005319,-0.0,-0.007549,-0.013033,-0.0
